In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing

In [41]:
import re

In [47]:
import collections

In [12]:
%load_ext autoreload
%autoreload 1
%matplotlib inline


In [105]:
import matplotlib
import matplotlib.pyplot as plt

In [23]:
in_mat_df_file = 'Biomarkers_snpnet_PRSs.tsv.gz'
out_PC_df_file = 'Biomarkers_snpnet_PRSs.PCs.tsv.gz'

In [3]:
in_mat_df = pd.read_csv(in_mat_df_file, compression='gzip', sep='\t')
in_mat = in_mat_df.values[:, 1:]


In [4]:
X = preprocessing.scale(in_mat)


In [5]:
print(in_mat.mean(axis=0))
print(in_mat.std(axis=0))

[ 5.47131788e-05  2.13818474e-04  2.75076985e-05 -1.85793465e-04
 -2.76534141e-07  1.74533224e-02  6.05582712e-04  1.63604858e-04
  4.18545137e-04 -1.19995573e-05  1.23069446e-03 -1.10402007e-04
 -2.69858855e-05  1.97365189e-05  5.19742793e-05  2.67570627e-06
 -2.08846848e-06  1.61833486e-04  7.25067979e-05 -1.61828225e-05
  8.09373967e-07 -3.63699028e-06  1.97930433e-04 -1.31266074e-04
  2.01153314e-06  6.01385977e-04  6.25754670e-05  2.97326360e-06
 -2.35818886e-05 -4.81005114e-06  1.91332754e-03 -1.52563505e-06
  4.30386855e-04 -4.24820848e-06 -2.74286284e-05 -8.84054790e-07
 -1.88411442e-04  8.79229823e-06]
[8.27802605e-05 2.49699851e-04 5.49199988e-05 1.27487565e-04
 2.14672991e-07 6.10665699e-02 3.99656085e-04 4.73625064e-04
 3.58443766e-04 2.72319007e-05 1.07684818e-03 8.66680510e-04
 3.77075498e-05 2.60870671e-05 1.92634700e-04 2.87349046e-06
 3.69657837e-06 1.83470085e-04 5.12831539e-05 1.46689098e-05
 1.07845629e-06 1.43598769e-05 1.58940940e-04 1.04259546e-04
 1.76722987e-06

In [6]:
print(X.mean(axis=0))
print(X.std(axis=0))

[-1.45353662e-16  2.39389915e-16  1.91149444e-17 -4.22984004e-16
 -7.45546056e-16 -5.60381887e-17 -1.45248287e-16 -1.56291541e-16
  1.69357986e-16 -1.38325179e-16 -1.81518209e-16 -4.25080957e-17
  2.00211656e-17  2.47598593e-16 -4.71867260e-17  1.56734114e-16
 -8.38280933e-17 -3.60971078e-16 -4.20233728e-17  4.17198940e-16
  1.23140706e-16  1.09715987e-16 -4.50507837e-16 -1.15137508e-16
  2.62066520e-17  1.85901791e-16 -2.39252928e-16  2.36655446e-16
 -8.86832260e-17  7.18391033e-17  2.80296318e-17  1.96881820e-16
  1.67935429e-16 -7.37621889e-19  5.30218420e-17 -1.29821452e-17
  1.58335807e-16  1.22919419e-17]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
svd = TruncatedSVD(n_components=20, n_iter=20, random_state=20190602)
Z = svd.fit_transform(X)


In [8]:
svd.explained_variance_ratio_.sum()

0.7600300058885109

In [9]:
np.cumsum(svd.explained_variance_ratio_)

array([0.08169697, 0.14363705, 0.19840622, 0.25024629, 0.29920217,
       0.34161391, 0.38166153, 0.41884197, 0.45530472, 0.48909125,
       0.522069  , 0.55249717, 0.58144927, 0.60837857, 0.63475922,
       0.6610008 , 0.68674069, 0.71154532, 0.7361408 , 0.76003001])

In [10]:
svd.explained_variance_ratio_

array([0.08169697, 0.06194008, 0.05476916, 0.05184007, 0.04895588,
       0.04241174, 0.04004762, 0.03718044, 0.03646274, 0.03378653,
       0.03297775, 0.03042817, 0.0289521 , 0.0269293 , 0.02638065,
       0.02624158, 0.02573989, 0.02480463, 0.02459548, 0.02388921])

In [11]:
Z.shape


(337151, 20)

In [21]:
Z_df = pd.DataFrame(
    Z, 
    columns=['PC{}'.format(x+1) for x in range(Z.shape[1])],
    index=list(in_mat_df['IID'])
)


In [24]:
Z_df.to_csv(
    out_PC_df_file,
    sep='\t', index=True, compression='gzip'
)

In [25]:
1

1

In [26]:
svd

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=20,
       random_state=20190602, tol=0.0)

In [54]:
tsvd_factorization = collections.OrderedDict()
tsvd_factorization['Sigma'] = svd.singular_values_[np.newaxis, :]
tsvd_factorization['V'] = svd.components_.T
tsvd_factorization['U'] = Z / svd.singular_values_


In [55]:
for k, v in tsvd_factorization.items():
    print(k, v.shape)

V (38, 20)
U (337151, 20)
Sigma (1, 20)


In [61]:
tsvd_factorization_idx = collections.OrderedDict()
tsvd_factorization_idx['Sigma'] = ['Sigma']
tsvd_factorization_idx['V'] = list(in_mat_df.columns[1:])
tsvd_factorization_idx['U'] = list(in_mat_df['IID'])


In [62]:
for k, v in tsvd_factorization.items():
    print(k, v.shape)
    pd.DataFrame(
        v, 
        columns=['PC{}'.format(x+1) for x in range(v.shape[1])],
        index=tsvd_factorization_idx[k]
    ).to_csv(
        re.sub('PCs', k, out_PC_df_file),
        sep='\t', index=True, compression='gzip'
    )


V (38, 20)
U (337151, 20)
Sigma (1, 20)


In [69]:
np.dot(V.T, V).shape

(20, 20)

In [78]:
np.diag(np.dot(V.T, V))

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [81]:
np.max(np.abs(np.triu(np.dot(V.T, V), 1))), np.median(np.triu(np.dot(V.T, V), 1))

(8.123646850574592e-16, 0.0)

In [74]:
VS = V * Sigma[0]

In [89]:
VS_norm = VS / np.sqrt(np.sum(VS ** 2, axis=1))[:, np.newaxis]

In [102]:
cos_sim = np.dot(VS_norm, VS_norm.T)

In [103]:
cos_sim

array([[ 1.        ,  0.98366573, -0.01582352, ...,  0.00357159,
         0.00808126,  0.00673321],
       [ 0.98366573,  1.        ,  0.00814278, ...,  0.04479318,
         0.00948595, -0.00669304],
       [-0.01582352,  0.00814278,  1.        , ...,  0.09381104,
         0.03748517, -0.03090798],
       ...,
       [ 0.00357159,  0.04479318,  0.09381104, ...,  1.        ,
        -0.39825633,  0.06109249],
       [ 0.00808126,  0.00948595,  0.03748517, ..., -0.39825633,
         1.        , -0.00429078],
       [ 0.00673321, -0.00669304, -0.03090798, ...,  0.06109249,
        -0.00429078,  1.        ]])

In [111]:
cos_sim_df = pd.DataFrame(
    cos_sim, 
    columns=tsvd_factorization_idx['V'],
    index=tsvd_factorization_idx['V']    
)

In [114]:
cos_sim_df.to_csv(
    re.sub('PCs', 'cos_sim', out_PC_df_file),
    sep='\t', index=True, compression='gzip'
)